# Word and POS tags embeddings


## Description
In this notebook, we are trying to parse $*.conllu$ files and extract the word embeddings, as well as the POS tags emeddings.

## Work to be done:
1. Learn how to parse $*.conllu$ files
2. Find a way to get all words embeddings.
3. Find a way to get all POS tags embeddings.

### 1. Read $*.conllu$ files

#### Word class:

In [1]:
class Word:
    """Word class that maps all details for a word from a *.conllu file. """
    
    def __init__(self, ID, FORM, LEMMA, UPOSTAG, XPOSTAG, FEATS, HEAD, DEPREL, DEPS, MISC):
        self.ID = ID           # Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes.
        self.FORM = FORM       # Word form or punctuation symbol.
        self.LEMMA  = LEMMA    # Lemma or stem of word form.
        self.UPOSTAG = UPOSTAG # Universal part-of-speech tag.
        self.XPOSTAG = XPOSTAG # Language-specific part-of-speech tag; underscore if not available.
        self.FEATS = FEATS     # List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
        self.HEAD = HEAD       # Head of the current word, which is either a value of ID or zero (0).
        self.DEPREL = DEPREL   # Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
        self.DEPS = DEPS       # Enhanced dependency graph in the form of a list of head-deprel pairs.
        self.MISC = MISC       # Any other annotation.
    
    def __str__(self):
        string = ''
        
        string += self.ID      if self.ID      != None else '_'
        string += " "
        string += self.FORM    if self.FORM    != None else '_'
        string += " "
        string += self.LEMMA   if self.LEMMA   != None else '_'
        string += " "
        string += self.UPOSTAG if self.UPOSTAG != None else '_'
        string += " "
        string += self.XPOSTAG if self.XPOSTAG != None else '_'
        string += " "
        string += self.FEATS   if self.FEATS   != None else '_'
        string += " "
        string += self.HEAD    if self.HEAD    != None else '_'
        string += " "
        string += self.DEPREL  if self.DEPREL  != None else '_'
        string += " "
        string += self.DEPS    if self.DEPS    != None else '_'
        string += " "
        string += self.MISC    if self.MISC    != None else '_'
        
        return string
        
    @staticmethod
    def from_line(line):
        tokens = line.split()
        
        assert len(tokens) == 10
        
        ID =      tokens[0] if tokens[0] != '_' else None
        FORM =    tokens[1] if tokens[1] != '_' else None
        LEMMA =   tokens[2] if tokens[2] != '_' else None
        UPOSTAG = tokens[3] if tokens[3] != '_' else None
        XPOSTAG = tokens[4] if tokens[4] != '_' else None
        FEATS =   tokens[5] if tokens[5] != '_' else None
        HEAD =    tokens[6] if tokens[6] != '_' else None
        DEPREL=   tokens[7] if tokens[7] != '_' else None
        DEPS =    tokens[8] if tokens[8] != '_' else None
        MISC =    tokens[9] if tokens[9] != '_' else None
        
        return Word(ID, FORM, LEMMA, UPOSTAG, XPOSTAG, FEATS, HEAD, DEPREL, DEPS, MISC)
        

#### Test Word class:

In [2]:
test_text = "4	proves	prove	VERB	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	0:root	_"
test_word = Word.from_line(test_text)
print(test_word)
print(test_word.FORM)

4 proves prove VERB VBZ Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 0 root 0:root _
proves


#### Sentence class:

In [3]:
class Sentence:
    """Sentence class that maps all details for a sentence from a *.conllu file. """
    
    def __init__(self,newdoc_id, send_id, text, words):
        self.newdoc_id = newdoc_id
        self.send_id = send_id
        self.text = text
        self.words = words
        
    def __str__(self):
        string = ''
        if self.newdoc_id != None:
            string += "# newdoc id = "
            string += str(self.newdoc_id)
            string += "\n"
        string += "# send_id = "
        string += str(self.send_id)
        string += "\n"
        string += "# text = "
        string += str(self.text)
        string += "\n"
        sentence_length = len(self.words)
        for index, word in enumerate(self.words):
            string += str(word)
            if index < sentence_length - 1:
                string += "\n"
        return string
    
        
    @staticmethod
    def from_lines(lines):
        newdoc_id=''
        send_id = ''
        text=''
        words = []

        
        for line in lines:
            if line.startswith("#"): #misc properties
                prop = line.split("=")[1]
                if line.startswith("newdoc", 2):
                    newdoc_id = prop
                    continue
                if line.startswith("sent_id", 2):
                    send_id = prop
                    continue
                if line.startswith("text", 2):
                    text = prop
                    continue
                continue
            else:   # words
                words.append(Word.from_line(line))

        return Sentence(newdoc_id, send_id, text, words)

#### Test Sentence class:

In [4]:
lines = ["# sent_id = weblog-blogspot.com_gettingpolitical_20030906235000_ENG_20030906_235000-0003",
"# text = Today's incident proves that Sharon has lost his patience and his hope in peace.",
"1	Today	today	NOUN	NN	Number=Sing	3	nmod:poss	3:nmod:poss	SpaceAfter=No",
"2	's	's	PART	POS	_	1	case	1:case	_",
"3	incident	incident	NOUN	NN	Number=Sing	4	nsubj	4:nsubj	_",
"4	proves	prove	VERB	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	0:root	_",
"5	that	that	SCONJ	IN	_	8	mark	8:mark	_",
"6	Sharon	Sharon	PROPN	NNP	Number=Sing	8	nsubj	8:nsubj	_",
"7	has	have	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	8	aux	8:aux	_",
"8	lost	lose	VERB	VBN	Tense=Past|VerbForm=Part	4	ccomp	4:ccomp	_",
"9	his	he	PRON	PRP$	Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs	10	nmod:poss	10:nmod:poss	_",
"10	patience	patience	NOUN	NN	Number=Sing	8	obj	8:obj	_",
"11	and	and	CCONJ	CC	_	13	cc	13:cc	_",
"12	his	he	PRON	PRP$	Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs	13	nmod:poss	13:nmod:poss	_",
"13	hope	hope	NOUN	NN	Number=Sing	10	conj	10:conj	_",
"14	in	in	ADP	IN	_	15	case	15:case	_",
"15	peace	peace	NOUN	NN	Number=Sing	13	nmod	13:nmod	SpaceAfter=No",
"16	.	.	PUNCT	.	_	4	punct	4:punct	_"]
sentence = Sentence.from_lines(lines)
print(str(sentence), "\n")
print(sentence.text, "\n")
print(sentence.words[0], "\n")

# newdoc id = 
# send_id =  weblog-blogspot.com_gettingpolitical_20030906235000_ENG_20030906_235000-0003
# text =  Today's incident proves that Sharon has lost his patience and his hope in peace.
1 Today today NOUN NN Number=Sing 3 nmod:poss 3:nmod:poss SpaceAfter=No
2 's 's PART POS _ 1 case 1:case _
3 incident incident NOUN NN Number=Sing 4 nsubj 4:nsubj _
4 proves prove VERB VBZ Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 0 root 0:root _
5 that that SCONJ IN _ 8 mark 8:mark _
6 Sharon Sharon PROPN NNP Number=Sing 8 nsubj 8:nsubj _
7 has have AUX VBZ Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 8 aux 8:aux _
8 lost lose VERB VBN Tense=Past|VerbForm=Part 4 ccomp 4:ccomp _
9 his he PRON PRP$ Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs 10 nmod:poss 10:nmod:poss _
10 patience patience NOUN NN Number=Sing 8 obj 8:obj _
11 and and CCONJ CC _ 13 cc 13:cc _
12 his he PRON PRP$ Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs 13 nmod:poss 13:nmod:poss _
13 

In [5]:
en_path = "Annotated training data/UD_English-master/"
en_dev_filename = "en-ud-dev.conllu"
en_test_filename = "en-ud-test.conllu"
en_train_filename = "en-ud-train.conllu"

ro_path = "Annotated training data/UD_Romanian-dev/"
ro_dev_filename = "ro-ud-dev.conllu"
ro_test_filename = "ro-ud-test.conllu"
ro_train_filename = "ro-ud-train.conllu"

def read_sentences(path, filename):
    sentences = []
    sentence_lines = []
    file = open(path + filename, 'r')
    for line in file:
        if line == '\n':
            sentences.append(Sentence.from_lines(sentence_lines))
            sentence_lines = []
        else:
            sentence_lines.append(line)
            
    return sentences


en_train_sentences = read_sentences(en_path,en_train_filename)
en_dev_sentences = read_sentences(en_path,en_dev_filename)
en_test_sentences = read_sentences(en_path,en_test_filename)

ro_train_sentences = read_sentences(ro_path,ro_train_filename)
ro_dev_sentences = read_sentences(ro_path,ro_dev_filename)
ro_test_sentences = read_sentences(ro_path,ro_test_filename)

assert len(en_dev_sentences) == 2002# there are 2002 sentences in the english development dataset.


### 2. Extract words/POS tags and then extract embeddings

#### Imports

In [6]:
import numpy as np
import pickle
from collections import defaultdict, Counter
from random import random
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

In [7]:
def get_Gensim_sentences(sentences):
    """
    This function receives a list of Sencences and return 
    the word and POS sentences in the format Gensim needs, 
    as well as Counters for both words and POS tags
    Format: [['I', 'like', 'custard'],...]
    """
    gensim_word_sentences = []
    word_counts = Counter()
    gensim_POS_sentences = []
    POS_counts = Counter()
    for sentence in sentences:
        gensim_word_sentence = []
        gensim_POS_sentence = []
        for word in sentence.words:
            gensim_word_sentence.append(word.FORM)
            word_counts[word.FORM] += 1
            gensim_POS_sentence.append(word.UPOSTAG)
            POS_counts[word.UPOSTAG] += 1
        gensim_word_sentences.append(gensim_word_sentence)
        gensim_POS_sentences.append(gensim_POS_sentence)
    return gensim_word_sentences, word_counts, gensim_POS_sentences, POS_counts

In [8]:
# Get word sentences, word count and word vocabulary
gensim_word_sentences_train, word_counts_train, gensim_POS_sentences_train, POS_counts_train = get_Gensim_sentences(en_test_sentences)
gensim_word_sentences_dev, word_counts_dev, gensim_POS_sentences_dev, POS_counts_dev = get_Gensim_sentences(en_dev_sentences)
gensim_word_sentences_test, word_counts_test, gensim_POS_sentences_test, POS_counts_test = get_Gensim_sentences(en_test_sentences)

gensim_word_sentences = []
gensim_word_sentences.extend(gensim_word_sentences_train)
gensim_word_sentences.extend(gensim_word_sentences_dev)
gensim_word_sentences.extend(gensim_word_sentences_test)
gensim_word_count = word_counts_train + word_counts_dev + word_counts_test

word_vocabulary = gensim_word_count.keys()

# Get POS tag sentences, POS tag count and POS tag vocabulary
gensim_POS_sentences = []
gensim_POS_sentences.extend(gensim_POS_sentences_train)
gensim_POS_sentences.extend(gensim_POS_sentences_dev)
gensim_POS_sentences.extend(gensim_POS_sentences_test)
gensim_POS_count = POS_counts_train + POS_counts_dev + POS_counts_test

pos_tag_vocabulary = gensim_POS_count.keys()

### a. Gensim Word2Vector trained on our data:

In [9]:
gensim_word_model = Word2Vec(gensim_word_sentences, size=100, window=5, min_count=1, workers=4)
gensim_POS_model = Word2Vec(gensim_POS_sentences, size=100, window=3, min_count=1, workers=4)

In [10]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import d3
from bokeh.io import output_notebook
output_notebook()

def emb_scatter(data, names, N=20, perplexity=30.0):
    """
    Function for plotting embeddings and words using TSNE.
    TSNE finds a way to plot multidimensional data to a 
    bidimensional plane. It assures that data close in the 
    multidimensionalspace will also be close in budimensional 
    place, but not the other way around.
    """
    ## try to find some clusters ##
    print("finding clusters")
    kmeans = KMeans(n_clusters=N)
    kmeans.fit(data)
    klabels = kmeans.labels_

    ## get a tsne fit ##
    print("fitting tsne")
    tsne = TSNE(n_components=2, perplexity=perplexity)
    emb_tsne = tsne.fit_transform(data)
    
    ## plot the tsne of the embeddings with bokeh ##
    # source: https://github.com/oxford-cs-deepnlp-2017/practical-1
    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="T-SNE for most common words")

    # set colormap as a list
    colormap = d3['Category20'][N]
    colors = [colormap[i] for i in klabels]

    source = ColumnDataSource(data=dict(x1=emb_tsne[:,0],
                                        x2=emb_tsne[:,1],
                                        names=names,
                                        colors=colors))

    p.scatter(x="x1", y="x2", size=8, source=source, color='colors')

    labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')
    p.add_layout(labels)

    show(p)

Loading BokehJS ...

In [11]:
top_words = [k for k,v in gensim_word_count.most_common(1000)]
top_word_vecs = gensim_word_model[top_words]
emb_scatter(top_word_vecs, top_words, N=20)

finding clusters
fitting tsne


In [12]:
top_POS = [k for k,v in gensim_POS_count.most_common(1000)]
top_POS_vecs = gensim_POS_model[top_POS]
emb_scatter(top_POS_vecs, top_POS, N=17)

finding clusters
fitting tsne


### b. Gensim Word2Vector trained on Wikipedia data:

### c. GloVe word embeddings from  spacy library(300d):

In [27]:
import spacy
glove = spacy.load('en')

word_embeddings_dict = {}
for word in word_vocabulary:
    word_embeddings_dict[word] = glove(word).vector

POS_embeddings_dict = {}
for POS_tag in pos_tag_vocabulary:
    POS_embeddings_dict[POS_tag] = glove(POS_tag).vector

In [53]:
most_common_words = gensim_word_count.most_common(3000)
most_common_words_vectors = []
for word, count in most_common_words:
    most_common_words_vectors.append(word_embeddings_dict[word])

emb_scatter(most_common_words_vectors, [word for word, count in most_common_words], N=20)

finding clusters
fitting tsne


In [61]:
pos_vectors = []
for pos_tag in pos_tag_vocabulary:
    pos_vectors.append(POS_embeddings_dict[pos_tag])
emb_scatter(pos_vectors, list(pos_tag_vocabulary), N=17)

finding clusters
fitting tsne


### d. GloVe word embeddings from pre-trained word vectors(50d/100d):

In [73]:
# you need to download the file(s) from https://github.com/stanfordnlp/GloVe
file = open("GloveEmbeddings/glove.6B.50d.txt")
tokens = []
for line in file:
    tokens.append(line.split()[0])
    
unknown_words = []
for word in word_vocabulary:
    if word.lower() not in tokens:
        unknown_words.append(word)

### e. GloVe word embeddings from pre-trained word vectors(50d/100d):
